In [ ]:
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

def scrape_afcae_total():
    scraper = cloudscraper.create_scraper(
        browser={'browser': 'chrome', 'platform': 'windows', 'desktop': True}
    )
    all_films = []
    total_pages = 355 


    for page in range(0, total_pages):
        url = f"https://www.art-et-essai.org/les-films-recommandes?page={page}"
        
        try:
            # Tentative de récupération
            response = scraper.get(url, timeout=30)
            
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                table_body = soup.find('tbody')
                rows = table_body.find_all('tr')
                for row in rows:
                    # Extraction des colonnes
                    titre = row.find('td', class_='views-field-title')
                    directeur = row.find('td', class_='views-field-field-director-text')
                    date = row.find('td', class_='views-field-field-release-date')
                    distrib = row.find('td', class_='views-field-field-distributeur-text')
                    label = row.find('td', class_='views-field-field-label')
                    visa = row.find('td', class_='views-field-field-visa')

                    all_films.append({
                        'Titre': titre.get_text(strip=True) if titre else "N/A",
                        'Réalisateur': directeur.get_text(strip=True) if directeur else "N/A",
                        'Date_Sortie': date.get_text(strip=True) if date else "N/A",
                        'Distributeur': distrib.get_text(strip=True) if distrib else "N/A",
                        'Label': label.get_text(strip=True) if label else "N/A",
                        'VISA': visa.get_text(strip=True) if visa else "N/A",
                        'Source_Page': page
                    })
                
                # Feedback de progression
                if page % 10 == 0:
                    print(f"🚀 Progression : Page {page}/{total_pages} | {len(all_films)} films collectés...")

                # Sauvegarde intermédiaire toutes les 50 pages au cas où
                if page % 50 == 0 and page > 0:
                    temp_df = pd.DataFrame(all_films)
                    temp_df.to_csv(f"backup_page_{page}.csv", index=False, encoding='utf-8-sig')

            elif response.status_code == 403:
                print(f"Bloqué par le serveur à la page {page}. Pause de 30 secondes...")
                time.sleep(30)
                continue # On réessaie la même page
                
            else:
                print(f"Erreur HTTP {response.status_code} à la page {page}")
                
        except Exception as e:
            print(f"Erreur à la page {page}: {str(e)}")
            # Sauvegarde d'urgence en cas d'erreur
            if all_films:
                emergency_df = pd.DataFrame(all_films)
                emergency_df.to_csv(f"emergency_save_page_{page}.csv", index=False, encoding='utf-8-sig')
            continue
        
        # Pause aléatoire pour éviter de surcharger le serveur
        time.sleep(random.uniform(1, 3))
    
    # Export final
    
    if all_films:
        df_final = pd.DataFrame(all_films)
        
        # Nettoyage des doublons potentiels
        df_final = df_final.drop_duplicates(subset=['Titre', 'VISA'], keep='first')
        
        # Export CSV final
        output_file = "films_afcae_complet.csv"
        df_final.to_csv(output_file, index=False, encoding='utf-8-sig')
        
  
        return df_final
  

# Lancement du scraping
if __name__ == "__main__":
    df_films = scrape_afcae_total()

In [ ]:
#On check les la database
df_films.shape
df_films.info()
df_films.head()

In [ ]:
import pandas as pd
import gdown
import json
import os
import numpy as np

In [ ]:
# Configurer
YEAR_MIN = 1950  # Films depuis 1950
BLOCKBUSTER_VOTE_THRESHOLD = 2000  # Seuil pour blockbusters US

#Importer fichier TMDB
id_drive = '1VB5_gl1fnyBDzcIOXZ5vUSbCY68VZN1v'
output_tmdb = 'tmdb_final.csv'
url_drive = f'https://drive.google.com/uc?id={id_drive}'

#téléchargement de la base de donnée
gdown.download(url_drive, output_tmdb, quiet=False)

df_tmdb = pd.read_csv(output_tmdb)


# NETTOYAGE JSON

def clean_json(x):
    try:
        if pd.isna(x): return np.nan
        data = json.loads(x.replace("'", '"'))
        return ", ".join([i['name'] for i in data])
    except:
        return np.nan

if 'production_companies' in df_tmdb.columns:
    df_tmdb['companies_clean'] = df_tmdb['production_companies'].apply(clean_json)

# GARDE production_countries et le nettoie
if 'production_countries' in df_tmdb.columns:
    df_tmdb['countries_clean'] = df_tmdb['production_countries'].apply(clean_json)

# Suppression colonnes inutiles (GARDE production_countries)
cols_drop = ['homepage', 'video', 'backdrop_path', 'status', 'production_companies']
df_tmdb = df_tmdb.drop(columns=[c for c in cols_drop if c in df_tmdb.columns])


In [ ]:
# PARTIE IMDB 
# IMDB BASIC

print(f"\n📥 IMDb Basics : Sélection (>= {YEAR_MIN})...")
url_basics = "https://datasets.imdbws.com/title.basics.tsv.gz"
chunks_basics = []

if 'imdb_id' in df_tmdb.columns:
    ids_tmdb = set(df_tmdb['imdb_id'].dropna())
else:
    ids_tmdb = set()

with pd.read_csv(url_basics, sep='\t', compression='gzip',
                 usecols=['tconst', 'titleType', 'startYear', 'primaryTitle', 'genres', 'runtimeMinutes'],
                 chunksize=500000) as reader:
    for chunk in reader:
        chunk['startYear'] = pd.to_numeric(chunk['startYear'], errors='coerce')
        
        mask = (
            (chunk['titleType'] == 'movie') &
            (chunk['startYear'] >= YEAR_MIN) &
            (chunk['tconst'].isin(ids_tmdb))
        )
        
        filtered_chunk = chunk[mask]
        
        if not filtered_chunk.empty:
            chunks_basics.append(filtered_chunk[['tconst', 'primaryTitle', 'startYear', 'genres', 'runtimeMinutes']])

df_basics = pd.concat(chunks_basics, ignore_index=True)
print(f"   ✅ Films retenus ({YEAR_MIN}-2025) : {len(df_basics):,}")

ids_films_finaux = set(df_basics['tconst'])


In [ ]:
# IMDb NAME.BASICS (Noms des personnes)

print("\n📥 IMDb name.basics : Chargement des noms...")
url_names = "https://datasets.imdbws.com/name.basics.tsv.gz"

# Charge toute la base (peut prendre 5-10 min)
df_names = pd.read_csv(url_names, sep='\t', compression='gzip', low_memory=False)
print(f"   ✅ {len(df_names):,} personnes chargées")


In [ ]:
# IMDb PRINCIPALS (Directors, Actors, Actresses)


print("\n📥 IMDb title.principals : Extraction du cast & crew...")
url_principals = "https://datasets.imdbws.com/title.principals.tsv.gz"

chunks_directors = []
chunks_actors = []
chunks_actresses = []
chunk_count = 0

with pd.read_csv(url_principals, sep='\t', compression='gzip',
                 usecols=['tconst', 'ordering', 'nconst', 'category', 'characters'],
                 chunksize=500000) as reader:
    
    for chunk in reader:
        chunk_count += 1
        if chunk_count % 5 == 0:
            print(f"   📦 Chunk {chunk_count}...")
        
        mask = chunk['tconst'].isin(ids_films_finaux)
        chunk_filtered = chunk[mask]
        
        if chunk_filtered.empty:
            continue
        
        # REALISATEURS
        directors = chunk_filtered[chunk_filtered['category'] == 'director'][['tconst', 'nconst']]
        if not directors.empty:
            chunks_directors.append(directors)
        
        # ACTEURS
        actors = chunk_filtered[chunk_filtered['category'] == 'actor'][['tconst', 'ordering', 'nconst', 'characters']]
        if not actors.empty:
            chunks_actors.append(actors)
        
        # ACTRICES
        actresses = chunk_filtered[chunk_filtered['category'] == 'actress'][['tconst', 'ordering', 'nconst', 'characters']]
        if not actresses.empty:
            chunks_actresses.append(actresses)

In [ ]:
# TRAITEMENT REALISATEUR
if chunks_directors:
    print("\n👨‍🎬 Traitement des réalisateurs...")
    df_directors = pd.concat(chunks_directors, ignore_index=True)
    df_directors = df_directors.drop_duplicates(subset='tconst', keep='first')
    
    # Merge avec les noms
    df_directors = df_directors.merge(
        df_names[['nconst', 'primaryName', 'birthYear']],
        on='nconst',
        how='left'
    )
    
    df_directors.rename(columns={
        'nconst': 'director_id',
        'primaryName': 'director_name',
        'birthYear': 'director_birth'
    }, inplace=True)
    
    # Ajoute à df_basics
    df_basics = df_basics.merge(
        df_directors[['tconst', 'director_id', 'director_name', 'director_birth']],
        on='tconst',
        how='left'
    )


In [ ]:
# TRAITEMENT ACTREURS
if chunks_actors:
    df_actors = pd.concat(chunks_actors, ignore_index=True)
    df_actors = df_actors.sort_values(['tconst', 'ordering']).groupby('tconst').head(5)
    
    df_actors = df_actors.merge(
        df_names[['nconst', 'primaryName']],
        on='nconst',
        how='left'
    )
    
    actors_agg = df_actors.groupby('tconst').agg({
        'nconst': lambda x: '|'.join(x.astype(str)),
        'primaryName': lambda x: '|'.join(x.dropna().astype(str))
    }).reset_index()
    
    actors_agg.rename(columns={
        'nconst': 'actors_ids',
        'primaryName': 'actors_names'
    }, inplace=True)
    
    df_basics = df_basics.merge(actors_agg, on='tconst', how='left')

In [ ]:
# TRAITEMENT ACTRICES
if chunks_actresses:
    df_actresses = pd.concat(chunks_actresses, ignore_index=True)
    df_actresses = df_actresses.sort_values(['tconst', 'ordering']).groupby('tconst').head(5)
    
    df_actresses = df_actresses.merge(
        df_names[['nconst', 'primaryName']],
        on='nconst',
        how='left'
    )
    
    actresses_agg = df_actresses.groupby('tconst').agg({
        'nconst': lambda x: '|'.join(x.astype(str)),
        'primaryName': lambda x: '|'.join(x.dropna().astype(str))
    }).reset_index()
    
    actresses_agg.rename(columns={
        'nconst': 'actresses_ids',
        'primaryName': 'actresses_names'
    }, inplace=True)
    
    df_basics = df_basics.merge(actresses_agg, on='tconst', how='left')

In [ ]:
# FUSION FINALE AVEC TMDB
df_final = pd.merge(df_basics, df_tmdb, left_on='tconst', right_on='imdb_id', how='inner')
df_final = df_final.drop(columns=['imdb_id'], errors='ignore')

# Nettoie et combine la durée
df_final['runtimeMinutes'] = pd.to_numeric(df_final['runtimeMinutes'], errors='coerce')
if 'runtime' in df_final.columns:
    df_final['runtime'] = pd.to_numeric(df_final['runtime'], errors='coerce')
    df_final['duration'] = df_final['runtime'].fillna(df_final['runtimeMinutes'])
else:
    df_final['duration'] = df_final['runtimeMinutes']



In [ ]:
# FILTRAGE PAR CATÉGORIE : Comédies FR + Blockbusters US

# COMÉDIES FRANÇAISES
mask_comedies_fr = (
    df_final['production_companies_country'].str.contains('FR', na=False, case=False) &
    df_final['genres_x'].str.contains('Comedy', na=False, case=False)
)
df_comedies_fr = df_final[mask_comedies_fr].copy()
df_comedies_fr['category'] = 'Comédie Française'

# BLOCKBUSTERS AMÉRICAINS
mask_blockbusters_us = (
    df_final['production_companies_country'].str.contains('US', na=False, case=False) &
    (df_final['vote_count'] >= BLOCKBUSTER_VOTE_THRESHOLD)
)
df_blockbusters_us = df_final[mask_blockbusters_us].copy()
df_blockbusters_us['category'] = 'Blockbuster Américain'

# Combine les deux catégories
df_final_filtered = pd.concat([df_comedies_fr, df_blockbusters_us], ignore_index=True)


In [ ]:
# EXPORT

# Export principal (tous les films)
df_final.to_csv("Dataset_1960_Plus.csv", index=False, encoding='utf-8-sig')

# Export filtré (comédies FR + blockbusters US)
df_final_filtered.to_csv("Database_Comedies_FR_Blockbusters_US.csv", index=False, encoding='utf-8-sig')

# Exports séparés
df_comedies_fr.to_csv("Database_Comedies_Francaises.csv", index=False, encoding='utf-8-sig')

df_blockbusters_us.to_csv("Database_Blockbusters_Americains.csv", index=False, encoding='utf-8-sig')




In [ ]:
import pandas as pd
import numpy as np
import datetime

In [ ]:
# Importer les databases

df_tt = pd.read_csv(r"C:\Users\Consultant\Desktop\creuze_project\Dataset_1960_Plus.csv")
df_afcae = pd.read_csv(r"C:\Users\Consultant\Desktop\creuze_project\films_afcae_complet.csv")

In [ ]:
def fix_director_name(name):
    if pd.isna(name) or name == "":
        return name
    
    parts = str(name).split()
    if len(parts) >= 2:
        # On suppose que le premier mot est le NOM (ex: TAN) 
        # et le reste le Prénom (ex: Royston)
        nom = parts[0].capitalize()
        prenom = " ".join(parts[1:]).title()
        return f"{prenom} {nom}"
    return str(name).title()

# Application sur ta database
df_afcae['director_clean'] = df_afcae['Réalisateur'].apply(fix_director_name)

In [ ]:
def normalize_names(name):
    if pd.isna(name): return ""
    # On met en minuscule, on enlève les accents et on sépare les mots
    parts = str(name).lower().strip().split()
    # On trie les mots par ordre alphabétique : ['royston', 'tan'] ou ['tan', 'royston'] deviennent identiques
    parts.sort()
    return "".join(parts)

# Creation d'une colonne avec les noms normalisés
df_afcae['dir_key'] = df_afcae['Réalisateur'].apply(normalize_names)
df_tt['dir_key'] = df_tt['director_name'].apply(normalize_names)

In [ ]:
# Pour AFCAE : on convertit et on gère les erreurs
df_afcae['release_date'] = pd.to_datetime(df_afcae['Date_Sortie'], dayfirst=True, errors='coerce')

# Pour TMDB
df_tt['release_date'] = pd.to_datetime(df_tt['release_date'], errors='coerce')

# On le remet au format texte, mais version TMDB (1962-06-22)
df_afcae['release_date'] = df_afcae['release_date'].dt.strftime('%Y-%m-%d')

# --- 2. HARMONISATION DES TITRES ---
# On met tout en minuscules (.str.lower()) et on enlève les espaces inutiles (.str.strip())
df_afcae['titre_key'] = df_afcae['Titre'].str.capitalize().str.strip()
df_tt['titre_key'] = df_tt['original_title'].str.capitalize().str.strip()

# Harmonisation des titres (Minuscules + Nettoyage)
df_afcae['titre_key'] = df_afcae['Titre'].str.lower().str.strip()
df_tt['titre_key'] = df_tt['original_title'].str.lower().str.strip()

# Harmonisation des dates (Année uniquement)
# On extrait l'année car le jour précis peut varier d'une base à l'autre
df_afcae['year_key'] = pd.to_datetime(df_afcae['Date_Sortie'], dayfirst=True, errors='coerce').dt.year
df_tt['year_key'] = pd.to_datetime(df_tt['release_date'], errors='coerce').dt.year

# Harmonisation des réalisateurs
df_afcae['dir_key'] = df_afcae['director_clean'].str.lower().str.strip()
df_tt['dir_key'] = df_tt['director_name'].str.lower().str.strip()

# Comparer et rendre chaque caractere similaire
df_tt['dir_key'] = df_tt['director_name'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
df_afcae['dir_key'] = df_afcae['director_clean'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
df_tt['titre_key'] = df_tt['original_title'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
df_afcae['titre_key'] = df_afcae['Titre'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()

In [ ]:
#rassembler toutes les databases
df_art_final = pd.merge(
    df_afcae, 
    df_tt, 
    left_on=['titre_key', 'dir_key'],
    right_on=['titre_key', 'dir_key'],
    how='inner'
)


#Nettoyage
#Sélectionner les films suppérieur à 60 minutes
art_essai = df_art_final[(df_art_final['runtime'] >= 60) | (df_art_final['runtime'].isna())]

#ajouter les autres Databases
df_reste = pd.read_csv(r"C:\Users\Consultant\Desktop\creuze_project\Database_Comedies_FR_Blockbusters_US.csv")
df_final_complet = pd.concat([art_essai, df_reste], axis=0, ignore_index=True)

#Supprimer les colonnes inutiles
cols_to_drop = [
    'adult', 'budget', 'genres_y', 'release_date', 'title',
    'production_companies_name', 'production_companies_country',
    'countries_clean', 'tagline', 'runtimeMinutes', 'release_date_y', 'director_birth'
]

df_complet = df_final_complet.drop(columns=cols_to_drop, errors='ignore')

# Supprimer les doublons basés sur la colonne 'id'
df_complet = df_complet.drop_duplicates(subset=['id'], keep='first')

#Vérifier Valeur manquante
df_complet.isna().sum()

#5 Réalisateurs manquants
df_complet[df_complet['director_name'].isna()]
# On cible le film par son titre et on remplit le nom des réalisateurs
df_complet.loc[df_complet['primaryTitle'] == 'Ugly Melanie', 'director_name'] = 'Allan Mauduit'
df_complet.loc[df_complet['primaryTitle'] == 'The Shiny Shrimps', 'director_name'] = 'Mathias Le Goff'
df_complet.loc[df_complet['primaryTitle'] == 'Fake News', 'director_name'] = 'Mouloud Achour|Dominique Baumard'
df_complet.loc[df_complet['primaryTitle'] == 'The Adventures of Felix', 'director_name'] = "Olivier Ducastel|Jacques Martineau"
df_complet.loc[df_complet['primaryTitle'] == "Côte d'Azur", 'director_name'] = "Olivier Ducastel|Jacques Martineau"

#remplacer valeur manquante dans Catégorie 
df_complet['category']= df_complet['category'].fillna('Art et Essai')

#remplacer valeur manquante dans le casting
cols = ['actors_ids', 'actors_names', 'actresses_ids', 'actresses_names']
df_complet[cols] = df_complet[cols].fillna('UNKNOWN')
df_complet['director_id'] = df_complet['director_id'].fillna('UNKNOWN')

#remplacer les synopsis manquants par inconnu
df_complet['overview'] = df_complet['overview'].fillna('UNKNOWN')

#remplacer les valeurs manquantes des affiches
df_complet['poster_path'] = df_complet['poster_path'].fillna('UNKNOWN')

#remplacer les valeurs manquantes des Distributeurs
df_complet['Distributeur'] = df_complet['Distributeur'].fillna('UNKNOWN')

#Retirer les colonnes en trop pour les statistiques

cols_to_drop = [
    'director_id','actors_ids', 'actresses_ids', 'id', 'spoken_languages', 'revenue',
    'production_countries', 'poster_path'
]

df_stat = df_complet.drop(columns=cols_to_drop, errors='ignore')

#changement des noms des colonnes
df_copy = df_complet.copy()
df_copy = df_copy.rename(columns={'startYear': 'Année_de_Sortie',
                       'genres_x': 'Genre',
                       'director_id': 'Réalisateur_id',
                       'director_name' : 'Réalisateur',
                       'actors_ids': 'Acteur_Id',
                       'actors_names': 'Acteur',
                       'actresses_ids': 'Actrice_Id',
                       'actresses_names': 'Actrice',
                       'id': 'Id_film', 
                       'spoken_languages': 'VO',
                       'original_title' : 'Titre',
                       'overview': 'Synopsis',
                       'popularity': 'Popularité',
                       'revenue': 'Revenues',
                       'runtime': 'Durée',
                       'spoken_languages': 'Langues',
                       'vote_average': 'Note',
                       'vote_count': 'Nombre_de_Vote',
                       'production_countries': 'Pays', 
                       'poster_path': 'Affiche_de_Film' ,
                       'category': 'Catégorie'
                       },
               inplace=False)

cols_to_drop = [
    'primaryTitle','duration'
]

df= df_copy.drop(columns=cols_to_drop, errors='ignore')
df = df[df['Durée'] >= 60]
df.to_csv('Database_finale.csv', index=False, encoding='utf-8')